# Playing with the DataSet

### Creating Sample File

In [1]:
import pandas as pd
import random
import csv

In [2]:
bible_data = pd.read_csv('datasets/bible_data.csv')
size = len(bible_data)

### Creating sentence embedding

We are not going to care about efficiency, but just do it
Because there is already a library called SentenceTransformer, we are not going to be bothered using deBERTa. If we need to do more customizing, we can always do it later.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
import numpy as np
from tqdm import tqdm

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
model = SentenceTransformer('all-MiniLM-L12-v2')
embeddings = np.array(model.encode(bible_data['text'], show_progress_bar=True))

# saving the numbers so it can be loaded up easily later
np.savez('datasets/bible_embeddings.npz', embeddings=embeddings, ids=bible_data['id'].values)

/home/codespace/.local/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [6]:
print(-float(model.similarity(embeddings[0], embeddings[1])))
print(sample_data['citation'][1])

-0.3130233585834503
Leviticus 24:3


### Using Sentence Embedding to compare each other

In [7]:
import heapq
from collections import defaultdict

In [8]:
# we are going to store top 10 similar verses for each verse
# also save top 100 similar verses in a list

# make sure that a separate list is created for each verse
verses_top = defaultdict(list)
overall_top = []

print(verses_top)

embeddings = np.load('datasets/sample_bible_embeddings.npy')

defaultdict(<class 'list'>, {})


In [9]:
greatest_similarity = -1.0
smallest_similarity = 1.0

# comparing each verse with every other verse
for i in range(len(sample_data)):

    verse1 = sample_data['citation'][i]

    for j in range(i + 1, len(sample_data)):
        similarity = float(model.similarity(embeddings[i], embeddings[j]))

        verse2 = sample_data['citation'][j]

        greatest_similarity = max(greatest_similarity, similarity)
        smallest_similarity = min(smallest_similarity, similarity)

        # we keep adding numbers and removing the smallest similarity
        item = (similarity, verse1, verse2)
        heapq.heappush(overall_top, item)

        if len(overall_top) > 10:
            heapq.heappop(overall_top)

        item = (similarity, verse2)
        heapq.heappush(verses_top[verse1], item)

        if len(verses_top[verse1]) > 10:
            heapq.heappop(verses_top[verse1])

        item = (similarity, verse1)
        heapq.heappush(verses_top[verse2], item)

        if len(verses_top[verse2]) > 10:
            heapq.heappop(verses_top[verse2])

        

In [10]:
import pandas as pd

In [12]:
# putting the results into a DataFrame
print(greatest_similarity)
print(smallest_similarity)


for item in reversed(sorted(overall_top)):
    print(item[0])
    print(f"{item[1]}  {sample_data.loc[sample_data['citation'] == item[1], 'text'].values[0]}")
    print(f"{item[2]}  {sample_data.loc[sample_data['citation'] == item[2], 'text'].values[0]}")
    print()


0.6882215142250061
-0.09599751234054565
0.6882215142250061
Galatians 5:5  For we through the Spirit wait for the hope of righteousness by faith. 

Psalms 33:20  Our soul waiteth for the LORD: he is our help and our shield. 


0.6505152583122253
Deuteronomy 11:27  A blessing, if ye obey the commandments of the LORD your God, which I command you this day: 

Jeremiah 7:23  But this thing commanded I them, saying, Obey my voice, and I will be your God, and ye shall be my people: and walk ye in all the ways that I have commanded you, that it may be well unto you. 


0.6193944215774536
Psalms 5:12  For thou, LORD, wilt bless the righteous; with favour wilt thou compass him as with a shield. 

Psalms 33:20  Our soul waiteth for the LORD: he is our help and our shield. 


0.6137830018997192
Deuteronomy 11:27  A blessing, if ye obey the commandments of the LORD your God, which I command you this day: 

Psalms 104:35  Let the sinners be consumed out of the earth, and let the wicked be no more. B